# Driverless AIモデルのMLOpsデプロイメント - Step1（AI Engine Manager接続）
- AI Engine Managerより、Driverless AIへ接続し、Projectの作成とExperimentの登録を実施

参考ワークフロー：https://docs.h2o.ai/mlops/py-client/py-client-examples/dai

In [9]:
import getpass

import h2o_authn 
print(h2o_authn.__version__)

import h2o_engine_manager
print(h2o_engine_manager.__version__)

1.1.1
0.6.0


In [5]:
#import driverlessai
#driverlessai.__version__

## Driverless AIで作成済みExperimentをProject登録

AI Engine ManagerとDriverless AIへの接続に関して：　https://github.com/yukismd/H2O_AI_Cloud/blob/main/client_access_external/Client_Access_ModelBuild.ipynb

### AI Engine Manager接続

AI Engine Managerに関して：https://h2oai.github.io/ai-engine-manager/

Python Client Example :  https://h2oai.github.io/ai-engine-manager/py/dai_examples

In [13]:
# AI Cloudへの認証
print("https://internal.dedicated.h2o.ai/auth/get-platform-token {}".format('よりトークンを取得'))
token_provider = h2o_authn.TokenProvider(
  refresh_token=getpass.getpass('Enter your platform token: '),
  client_id="hac-platform-public", 
  token_endpoint_url="https://auth.internal.dedicated.h2o.ai/auth/realms/hac/protocol/openid-connect/token" 
)

https://internal.dedicated.h2o.ai/auth/get-platform-token よりトークンを取得
Enter your platform token: ········


In [16]:
# AI Engine Managerへの接続
aiem = h2o_engine_manager.login(
  environment="https://internal.dedicated.h2o.ai", 
  token_provider=token_provider
)
aiem

### Driverless AIへの接続とProject作成とExperiment登録

In [46]:
# 既存インスタンスの確認
aiem.dai_engine_client.list_engines()

{'engines': [{'annotations': {},
 'api_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/syc',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7fd5886dac40>,
 'config': {},
 'cpu': 2,
 'create_time': datetime.datetime(2023, 12, 21, 5, 8, 28, tzinfo=tzutc()),
 'creator': 'users/bc34ef4d-ff29-4375-8bab-04d860fcac32',
 'creator_display_name': 'yuki.shimada@h2o.ai',
 'current_idle_duration': None,
 'current_running_duration': None,
 'delete_time': None,
 'display_name': 'syc',
 'engine_id': 'syc',
 'gpu': 0,
 'login_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/syc/oidc/login',
 'max_idle_duration': '1h',
 'max_running_duration': '8h',
 'memory_bytes': '4Gi',
 'name': 'workspaces/default/daiEngines/syc',
 'reconciling': False,
 'resume_time': datetime.datetime(2023, 12, 26, 2, 35, 5, tzinfo=tzutc()),
 'state': <DAIEngineState.STATE_PAUSED: 'STATE_PAUSED'>,
 'storage_bytes': '

In [47]:
# 作成しているインスタンスの台数
aiem.dai_engine_client.list_engines().total_size

2

In [48]:
# 1台目のDriverless AIの名前とID
aiem.dai_engine_client.list_engines().engines[0].display_name,  aiem.dai_engine_client.list_engines().engines[0].engine_id

('syc', 'syc')

In [49]:
# 2台目のDriverless AIの名前とID
aiem.dai_engine_client.list_engines().engines[1].display_name,  aiem.dai_engine_client.list_engines().engines[1].engine_id

('DAI-small', 'dai-small')

In [131]:
# 利用Driverless AI
dai_instance = aiem.dai_engine_client.get_engine('dai-small')   # IDを指定
dai_instance

{'annotations': {},
 'api_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/dai-small',
 'client_info': <h2o_engine_manager.clients.dai_engine.client_info.ClientInfo object at 0x7fd5886dac40>,
 'config': {},
 'cpu': 6,
 'create_time': datetime.datetime(2023, 11, 21, 6, 33, 58, tzinfo=tzutc()),
 'creator': 'users/bc34ef4d-ff29-4375-8bab-04d860fcac32',
 'creator_display_name': 'yuki.shimada@h2o.ai',
 'current_idle_duration': None,
 'current_running_duration': '568s',
 'delete_time': None,
 'display_name': 'DAI-small',
 'engine_id': 'dai-small',
 'gpu': 0,
 'login_url': 'https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/dai-small/oidc/login',
 'max_idle_duration': '1h',
 'max_running_duration': '8h',
 'memory_bytes': '24Gi',
 'name': 'workspaces/default/daiEngines/dai-small',
 'reconciling': False,
 'resume_time': datetime.datetime(2024, 1, 4, 5, 23, 21, tzinfo=tzutc()),
 'state': <DAIEngineState.STATE_RUNNING: 'STATE_RUNNING'

In [96]:
type(dai_instance)

h2o_engine_manager.clients.dai_engine.dai_engine.DAIEngine

In [98]:
# 停止しているDriverless AIを開始（Resume）する場合
dai_instance.resume()

In [134]:
# 利用Driverless AIのステータス確認
dai_instance.state

# 起動中（Running）であることを確認

<DAIEngineState.STATE_RUNNING: 'STATE_RUNNING'>

In [135]:
# 利用Driverless AIの取得（インスタンスの状態がrunningである必要あり）
dai = dai_instance.connect()
dai

<class 'driverlessai._core.Client'> https://enginemanager.internal.dedicated.h2o.ai/workspaces/default/daiEngines/dai-small

In [140]:
type(dai)

driverlessai._core.Client

#### 以下、Driverless AI Python Clientによる操作

Driverless AI Python Client: https://docs.h2o.ai/driverless-ai/pyclient/docs/html/index.html

In [136]:
# Experimentのリスト
dai.experiments.list()

    | Type       | Key                                  | Name
----+------------+--------------------------------------+----------------
  0 | Experiment | d621f974-9a28-11ee-9e32-5635106bd130 | protein-kd
  1 | Experiment | fe1beb2a-9a27-11ee-9e32-5635106bd130 | protein-af
  2 | Experiment | b437a73a-8f21-11ee-a023-361cd41ebbf9 | protein-KD
  3 | Experiment | 2449f79a-8daa-11ee-931e-de9862232f89 | titanic1
  4 | Experiment | 90f8451c-8ce8-11ee-ba2f-1ee91261fde9 | walmart
  5 | Experiment | 308fa33a-8cc2-11ee-bb5c-3e7c02aca797 | car
  6 | Experiment | 00ca3ca4-8cbe-11ee-bb5c-3e7c02aca797 | amazon-review
  7 | Experiment | 315c48e6-8839-11ee-93d6-eae6509eb991 | ucicc-default
  8 | Experiment | 5a381bf6-8838-11ee-93d6-eae6509eb991 | boston-default

In [138]:
# 各Experimentオブジェクトの取得
mod1 = dai.experiments.list()[3]
mod2 = dai.experiments.list()[6]

mod1.name, mod2.name, type(mod1)

('titanic1', 'amazon-review', driverlessai._experiments.Experiment)

In [139]:
# Projectのリスト
dai.projects.list()

    | Type    | Key                                  | Name
----+---------+--------------------------------------+-------------------
  0 | Project | 1da36031-b05f-47c0-b913-e170084581d8 | titanic
  1 | Project | 91d6b769-9e9b-4606-8984-278b632716d1 | Proj-iris
  2 | Project | 297142df-702f-4212-ad83-3bed0218be1f | proj_boston
  3 | Project | dfc7fd0a-7645-4b7e-a082-b8846b032daa | car_obj_detection

プロジェクト作成以降の参考：https://github.com/yukismd/H2O_AI_Cloud/blob/main/client_access_external/DAI_ModelDeploy-Step1.ipynb